In [ ]:
import numpy as np # type: ignore
import matplotlib.patches as patches # type: ignore
import matplotlib.pyplot as plt # type: ignore
from matplotlib.collections import LineCollection # type: ignore

In [ ]:
def is_point_in_zone(point, zone):
    x, y = point
    return (
        x >= zone["x"]
        and x <= zone["x"] + zone["width"]
        and y >= zone["y"]
        and y <= zone["y"] + zone["height"]
    )

# Precompute street points inside each zone
def get_street_points_by_zone(street_graph, zones):
    return {
        zone["id"]: [
            {"x": data["x"], "y": data["y"], "id": node_id}
            for node_id, data in street_graph.nodes(data=True)
            if is_point_in_zone((data["x"], data["y"]), zone)
        ]
        for zone in zones
    }


In [ ]:
def draw_on_lines(graph, lines, colour):
    # Fetch all node positions in one go
    pos = {node: (data["x"], data["y"]) for node, data in graph.nodes(data=True)}
    
    # Convert lines into an array of coordinates
    segments = np.array([(pos[u], pos[v]) for u, v in lines])

    # Use LineCollection for faster bulk rendering
    lc = LineCollection(segments, colors=colour, linewidths=2, alpha=1)
    plt.gca().add_collection(lc)

def add_route(graph, route, colour, is_actual_route=False):
    pos = {node: (data["x"], data["y"]) for node, data in graph.nodes(data=True)}

    start, end = pos[route[0]], pos[route[-1]]

    if is_actual_route:
        plt.scatter(*start, color="green", s=100, zorder=5)
        plt.scatter(*end, color="red", s=100, zorder=5)
    else:
        plt.scatter(*start, color="blue", s=100, zorder=5)
        plt.scatter(*end, color="orange", s=100, zorder=5)

    draw_on_lines(graph, zip(route[:-1], route[1:]), colour)

def visualise(street_graph, zones, width, height, trips=[]):
    plt.figure(figsize=(15, 10))

    plt.xlim(0, width)
    plt.ylim(0, height)
    plt.gca().set_facecolor("#f8f9fa")

    draw_on_lines(street_graph, street_graph.edges(), "#8080807f")

    for zone in zones:
        rect = patches.Rectangle(
            (zone["x"], zone["y"]),
            zone["width"],
            zone["height"],
            linewidth=1,
            edgecolor="#333333",
            facecolor=zone["colour"],
            alpha=0.5,
        )
        plt.gca().add_patch(rect)
        plt.text(
            zone["x"] + zone["width"] / 2,
            zone["y"] + zone["height"] / 2,
            zone["name"],
            ha="center",
            va="center",
            fontweight="bold",
        )

    for trip in trips:
        add_route(street_graph, trip["route"], "black", is_actual_route=True)

        if trip["estimated_route"] is not None:
            add_route(street_graph, trip["estimated_route"], "black", is_actual_route=False)

    plt.legend()
    plt.tight_layout()
    plt.show()
